In [ ]:
!pip install geopandas --quiet

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.patheffects as pe
import seaborn as sns

# Carregando os dados geográficos
brasil = gpd.read_file('https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/Brasil/BR/BR_UF_2022.zip')
america_sul = gpd.read_file('https://geonode.paranagua.pr.gov.br/geoserver/ows?srsName=EPSG%3A4674&outputFormat=json&service=WFS&request=GetFeature&typename=geonode%3Aa__031_001_americaDoSul')

# Carregando a camada adicional
municipios = gpd.read_file('G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\analysis\datasets\\vetorial_positivos.geojson')

# Lista de anos para os quais você tem dados
anos = [2015, 2016, 2017, 2018, 2019, 2022]

# Loop através de cada ano
for ano in anos:
    # Carregando seus dados
    gdf = gpd.read_file('G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\analysis\datasets\vetorial_positivos.geojson')

    # Calculando as coordenadas do centróide
    gdf['centroid'] = gdf.geometry.centroid
    gdf['X_Coord'] = gdf['centroid'].x
    gdf['Y_Coord'] = gdf['centroid'].y

    # Criando a figura e os eixos
    fig, ax = plt.subplots(1, 1)

    # Desenhando os mapas com as cores e espessuras de linha especificadas
    america_sul.plot(ax=ax, color='#c7c7c9', edgecolor='black', linewidth=0.26)
    brasil.plot(ax=ax, color='#f5f5e0', edgecolor='black', linewidth=0.26)
    municipios.plot(ax=ax, color='#f7d193', edgecolor='black', linewidth=0.26)

    # Normalizando os valores em "Positivos_ano"
    gdf[f"Positivos_{ano}_normalized"] = gdf[f"Positivos_{ano}"] / gdf[f"Positivos_{ano}"].max()

    # Plotando o mapa de círculos proporcionais
    ax.scatter(
        gdf["X_Coord"],
        gdf["Y_Coord"],
        s=gdf[f"Positivos_{ano}_normalized"] * 500,  # Ajuste o fator de escala conforme necessário
        c='red',  # Ajuste a cor dos círculos
        alpha=0.5  # Ajuste a transparência conforme necessário
    )

    # Ajustando os limites do gráfico para se basear na extensão de gdf com uma margem
    minx, miny, maxx, maxy = gdf.total_bounds
    ax.set_xlim(minx - 1, maxx + 1)
    ax.set_ylim(miny - 1, maxy + 1)

    # Adicionando a seta norte
    arrow = plt.Arrow(0.05, 0.95, 0, 0.05, width=0.015, transform=ax.transAxes, color='black')
    arrow.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_patch(arrow)
    plt.text(0.05, 0.90, 'N', transform=ax.transAxes)

    # Adicionando a primeira barra de escala
    scalebar1 = AnchoredSizeBar(ax.transData, 3, '300 km', 'lower right', pad=0.0, frameon=False, size_vertical=1, color='black')
    scalebar1.txt_label._text.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_artist(scalebar1)

    # Adicionando títulos aos eixos
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Salvando a figura
    plt.savefig(f'G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\analysis\datasets\\Positivos_{ano}.png')

    # Mostrando o gráfico
    plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.patheffects as pe
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Carregando os dados geográficos
brasil = gpd.read_file('https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/Brasil/BR/BR_UF_2022.zip')
america_sul = gpd.read_file('https://geonode.paranagua.pr.gov.br/geoserver/ows?srsName=EPSG%3A4674&outputFormat=json&service=WFS&request=GetFeature&typename=geonode%3Aa__031_001_americaDoSul')

# Carregando a camada adicional
municipios = gpd.read_file('G:\\.shortcut-targets-by-id\\1SG0a72RcY788yL9ZXQd9gzVVAs7GX0cR\\.CASOS IMPORTADOS\\CONSOLIDAÇÃO_IMPORTADOS\\vetorial_positivos.geojson')

# Lista de anos para os quais você tem dados
anos = [2015, 2016, 2017, 2018, 2019, 2022]

# Criando uma paleta de cores personalizada
cmap = LinearSegmentedColormap.from_list(name='verde_amarelo_vermelho', colors=['green', 'yellow', 'red'])

# Loop através de cada ano
for ano in anos:
    # Carregando seus dados
    gdf = gpd.read_file('G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\analysis\datasets\\vetorial_positivos.geojson')

    # Calculando as coordenadas do centróide
    gdf['centroid'] = gdf.geometry.centroid
    gdf['X_Coord'] = gdf['centroid'].x
    gdf['Y_Coord'] = gdf['centroid'].y

    # Criando a figura e os eixos
    fig, ax = plt.subplots(1, 1)

    # Desenhando os mapas com as cores e espessuras de linha especificadas
    america_sul.plot(ax=ax, color='#c7c7c9', edgecolor='black', linewidth=0.26)
    brasil.plot(ax=ax, color='#f5f5e0', edgecolor='black', linewidth=0.26)
    municipios.plot(ax=ax, color='#f7d193', edgecolor='black', linewidth=0.26)

    # Plotando o mapa de calor interpolado usando kdeplot
    sns.kdeplot(
        ax=ax,
        data=gdf,
        x="X_Coord",
        y="Y_Coord",
        weights=f"Positivos_{ano}",
        fill=True,
        cmap=cmap,  # Usando a paleta de cores personalizada
        alpha=0.5  # Ajuste a transparência conforme necessário
    )

    # Ajustando os limites do gráfico para se basear na extensão de gdf com uma margem
    minx, miny, maxx, maxy = gdf.total_bounds
    ax.set_xlim(minx - 1, maxx + 1)
    ax.set_ylim(miny - 1, maxy + 1)

    # Adicionando a seta norte
    arrow = plt.Arrow(0.05, 0.95, 0, 0.05, width=0.015, transform=ax.transAxes, color='black')
    arrow.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_patch(arrow)
    plt.text(0.05, 0.90, 'N', transform=ax.transAxes)

    # Adicionando a primeira barra de escala
    scalebar1 = AnchoredSizeBar(ax.transData, 3, '300 km', 'lower right', pad=0.0, frameon=False, size_vertical=1, color='black')
    scalebar1.txt_label._text.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_artist(scalebar1)

    # Adicionando títulos aos eixos
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Salvando a figura
    plt.savefig(f'G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\maps\map_outputs\Estimativa_de_densidade_de_Kernel_paleta_2_2017.png')

    # Mostrando o gráfico
    plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.patheffects as pe
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap

# Carregando os dados geográficos
brasil = gpd.read_file('https://geoftp.ibge.gov.br/organizacao_do_territorio/malhas_territoriais/malhas_municipais/municipio_2022/Brasil/BR/BR_UF_2022.zip')
america_sul = gpd.read_file('https://geonode.paranagua.pr.gov.br/geoserver/ows?srsName=EPSG%3A4674&outputFormat=json&service=WFS&request=GetFeature&typename=geonode%3Aa__031_001_americaDoSul')

# Carregando a camada adicional
municipios = gpd.read_file('G:\\.shortcut-targets-by-id\\1SG0a72RcY788yL9ZXQd9gzVVAs7GX0cR\\.CASOS IMPORTADOS\\CONSOLIDAÇÃO_IMPORTADOS\\vetorial_positivos.geojson')

# Lista de anos para os quais você tem dados
anos = [2015, 2016, 2017, 2018, 2019, 2022]

# Criando uma paleta de cores personalizada
cmap = LinearSegmentedColormap.from_list(name='custom', colors=['blue', 'cyan', 'yellow', 'red'])

# Loop através de cada ano
for ano in anos:
    # Carregando seus dados
    gdf = gpd.read_file('G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\analysis\datasets\vetorial_positivos.geojson')

    # Calculando as coordenadas do centróide
    gdf['centroid'] = gdf.geometry.centroid
    gdf['X_Coord'] = gdf['centroid'].x
    gdf['Y_Coord'] = gdf['centroid'].y

    # Criando a figura e os eixos
    fig, ax = plt.subplots(1, 1)

    # Desenhando os mapas com as cores e espessuras de linha especificadas
    america_sul.plot(ax=ax, color='#c7c7c9', edgecolor='black', linewidth=0.26)
    brasil.plot(ax=ax, color='#f5f5e0', edgecolor='black', linewidth=0.26)
    municipios.plot(ax=ax, color='#f7d193', edgecolor='black', linewidth=0.26)

    # Plotando o mapa de calor interpolado usando kdeplot
    sns.kdeplot(
        ax=ax,
        data=gdf,
        x="X_Coord",
        y="Y_Coord",
        weights=f"Positivos_{ano}",
        fill=True,
        cmap=cmap,  # Usando a paleta de cores personalizada
        alpha=0.5  # Ajuste a transparência conforme necessário
    )

    # Ajustando os limites do gráfico para se basear na extensão de gdf com uma margem
    minx, miny, maxx, maxy = gdf.total_bounds
    ax.set_xlim(minx - 1, maxx + 1)
    ax.set_ylim(miny - 1, maxy + 1)

    # Adicionando a seta norte
    arrow = plt.Arrow(0.05, 0.95, 0, 0.05, width=0.015, transform=ax.transAxes, color='black')
    arrow.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_patch(arrow)
    plt.text(0.05, 0.90, 'N', transform=ax.transAxes)

    # Adicionando a primeira barra de escala
    scalebar1 = AnchoredSizeBar(ax.transData, 3, '300 km', 'lower right', pad=0.0, frameon=False, size_vertical=1, color='black')
    scalebar1.txt_label._text.set_path_effects([pe.Stroke(linewidth=3, foreground='white'), pe.Normal()])
    ax.add_artist(scalebar1)

    # Adicionando títulos aos eixos
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Salvando a figura
    plt.savefig(f'G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\maps\map_outputs\Estimativa de densidade de Kernel - {ano}.png')

    # Mostrando o gráfico
    plt.show()